<a href="https://colab.research.google.com/github/tikendraw/caption-generator/blob/main/train_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
import sys
if 'google.colab' in sys.modules:
    os.system('git clone https://github.com/tikendraw/caption-generator.git -q')
    os.chdir('caption-generator')


if not os.path.exists('funcyou'):
	os.system('git clone https://github.com/tikendraw/funcyou -q')

os.system('pip install funcyou/.')

0

In [2]:
! pip install funcyou/.

'another model.py'		   image-caption-generator-deep-learning.ipynb
 caption-generator		   libdevice.10.bc
 captiongenerator_notebook.ipynb   local_config_cuda
 code.txt			   main.py
 config.py			   model.py
 data				   preprocessing.py
 embedding			   __pycache__
 funcyou			   train_notebook.ipynb
 get_data.py			   utils.py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./funcyou
  Preparing metadata (setup.py) ... done
  Created wheel for funcyou: filename=funcyou-1.0.0-py3-none-any.whl size=10314 sha256=a40acc3d7975b5366880b55d22760ab30d59b49808120268b70a11881d61929b
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xe43ouc/wheels/82/48/fc/74dcbb32089fecdd2b516b6b61928ed33c3e1ab25c126e9e9e
Successfully built funcyou
  Attempting uninstall: funcyou
    Found existing installation: funcyou 1.0.0
    Uninstalling funcyou-1.0.0:
      Successfully uninstalled funcyou-1.0.0


In [3]:

import numpy as np
import pandas as pd
import random, math
import tensorflow as tf
import glob
import shutil
from zipfile import ZipFile
import datetime
import sys
from functools import cache
from pathlib import Path
from tqdm import tqdm
import regex as re

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from sklearn.model_selection import train_test_split


from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocessing
from tensorflow.keras.layers import (
    TextVectorization, Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense, Attention, MultiHeadAttention, Flatten, Dropout,
    Concatenate, Activation, GlobalAveragePooling2D
    )
from tensorflow.keras.layers import LSTM, Embedding, Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow import keras
from tensorflow.keras.utils import array_to_img, img_to_array
import string
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, TensorBoard
from model import LearningRateDecayCallback, get_model, masked_acc, masked_loss
from preprocessing import preprocess_text, embedding_matrix_creater, mapper, clean_words, clean_df
from utils import create_model_checkpoint

from config import config

from get_data import download_dataset
from funcyou.dataset import download_kaggle_dataset

94668760/94668760 [==============================] - 1s 0us/step


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
seed_value = 12321
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)


BATCH_SIZE =    config.BATCH_SIZE
IMG_SIZE =      config.IMG_SIZE
CHANNELS =      config.CHANNELS
IMG_SHAPE =     config.IMG_SHAPE
MAX_LEN =       config.MAX_LEN
EPOCHS =        config.EPOCHS
LEARNING_RATE = config.LEARNING_RATE
UNITS =         config.UNITS
raw_caption_file =  config.raw_caption_file
caption_file =  config.caption_file
image_dir =     config.image_dir
glove_path =    config.glove_path
TEST_SIZE =     config.TEST_SIZE
VAL_SIZE=       config.VAL_SIZE
EMBEDDING_DIMENSION =   config.EMBEDDING_DIMENSION 

In [5]:
pathh = '/content/kaggle.json'

In [6]:
if 'google.colab' in sys.modules:
    download_dataset(pathh)

Downloading...
kaggle datasets download -d hsankesara/flickr-image-dataset


100B [01:05, 1.54B/s]


Downloaded.
Extracting...
Moving...
Done
Cleaned.


In [20]:
! rm -rf flickr30k_images

# Reading

In [7]:
df = pd.read_csv(caption_file)

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158914 entries, 0 to 158913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      158914 non-null  int64 
 1   image_name      158914 non-null  object
 2   comment_number  158914 non-null  int64 
 3   comment         158914 non-null  object
 4   word_length     158914 non-null  int64 
 5   image_path      158914 non-null  object
dtypes: int64(3), object(3)
memory usage: 7.3+ MB
None


,Unnamed: 0,image_name,comment_number,comment,word_length,image_path
0,0,1000092795.jpg,0,startseq Two young guys with shaggy hair look ...,17,input/flickr30k/images/1000092795.jpg
1,1,1000092795.jpg,1,startseq Two young White males are outside nea...,11,input/flickr30k/images/1000092795.jpg
2,2,1000092795.jpg,2,startseq Two men in green shirts are standing ...,11,input/flickr30k/images/1000092795.jpg
3,3,1000092795.jpg,3,startseq A man in a blue shirt standing in a g...,11,input/flickr30k/images/1000092795.jpg
4,4,1000092795.jpg,4,startseq Two friends enjoy time together endseq,7,input/flickr30k/images/1000092795.jpg


In [8]:
df = df[df.comment_number == 1]
print(df.shape)

(31783, 6)


# Tokenize

In [9]:



#tokenizer
tokenizer = TextVectorization(standardize=preprocess_text)
tokenizer.adapt(df['comment'])


word_to_id = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
id_to_word = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)


In [11]:



#GLOVE embedding
glove_api_command = 'kaggle datasets download -d watts2/glove6b50dtxt'
glove_url = 'https://www.kaggle.com/datasets/watts2/glove6b50dtxt'

if 'google.colab' in sys.modules:

    download_kaggle_dataset(glove_api_command)
    os.makedirs('embedding', exist_ok = True)
    shutil.move('glove6b50dtxt.zip', 'embedding/glove.6B.50d.zip',)


# creating embedding matrix
word_dict = {word: i for i, word in enumerate(tokenizer.get_vocabulary())}

# Creating embedding matrix
embedding_matrix = embedding_matrix_creater(EMBEDDING_DIMENSION, word_index=word_dict)

# # Saving embedding_matrix for further use
# np.save("./embedding/embedding_matrix.npy", embedding_matrix, allow_pickle=True)
# # compressing
# ZipFile("embedding_matrix.zip", mode="w").write(
#     "./embedding/embedding_matrix.npy")


# load image model
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=tf.keras.layers.Input(shape=IMG_SHAPE))

resnet.trainable = False


# Creating dataset
TEST_SIZE = config.TEST_SIZE
VAL_SIZE =  config.VAL_SIZE

train, val = train_test_split(
    df[['image_path', 'comment']],  test_size=VAL_SIZE, random_state=11)
train, test = train_test_split(
    train[['image_path', 'comment']],  test_size=TEST_SIZE, random_state=11)


train_data = tf.data.Dataset.from_tensor_slices((train.image_path, train.comment))
test_data = tf.data.Dataset.from_tensor_slices((test.image_path, test.comment))
val_data = tf.data.Dataset.from_tensor_slices((val.image_path, val.comment))


train_data = train_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data =   test_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data =     val_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# resnet_output_flattened_shape = 8*8*2048

print("Number of training samples: %d" %
      tf.data.experimental.cardinality(train_data))
print("Number of validation samples: %d" %
      tf.data.experimental.cardinality(val_data))
print("Number of test samples: %d" %
      tf.data.experimental.cardinality(test_data))

VOCAB_SIZE = tokenizer.vocabulary_size()
print("Vocabulary size: %d" % VOCAB_SIZE)


kaggle datasets download -d watts2/glove6b50dtxt


100B [00:02, 49.9B/s]


Number of training samples: 3586
Number of validation samples: 199
Number of test samples: 189
Vocabulary size: 7653


In [12]:
model = get_model(embedding_matrix, VOCAB_SIZE)
print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=masked_loss,
              metrics=[masked_acc, masked_loss])


# model = tf.keras.models.load_model('/content/drive/MyDrive/cap-gen/2023-05-14 08:04:36.705364-30.tf')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 50)       382650      ['input_4[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 50, 16),     4288        ['embedding[0][0]']              
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                              

In [13]:

# Example usage
initial_lr = LEARNING_RATE
decay_rate = -0.001
decay_steps = 2

decay_callback = LearningRateDecayCallback(initial_lr, decay_rate, decay_steps)

os.makedirs('log', exist_ok=True)
csv_logger = CSVLogger('./log/training.log')
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)


In [14]:
print(initial_lr)

0.01


In [15]:
EPOCHS = 10

print(len(train_data) // EPOCHS, len(val_data) // EPOCHS)

steps_per_epoch = int(0.1*(len(train_data) / EPOCHS))
validation_steps =  int(.2*(len(val_data) / EPOCHS))
print(steps_per_epoch, validation_steps)


1195 66
119 13


In [16]:
history = model.fit(train_data,
                    epochs=EPOCHS,
                    validation_data=val_data,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[
                        decay_callback,
                        csv_logger, create_model_checkpoint(model_name = 'capgen', save_dir = 'checkpoints', monitor = 'masked_acc')
                                ]
                    )

Epoch 1/3
119/119 [==============================] - ETA: 0s - loss: 20.1196 - masked_acc: 4.0578e-04 - masked_loss: 20.1200

119/119 [==============================] - 268s 2s/step - loss: 20.1196 - masked_acc: 4.0578e-04 - masked_loss: 20.1200 - val_loss: 21.1002 - val_masked_acc: 0.0000e+00 - val_masked_loss: 21.1075
Epoch 2/3
119/119 [==============================] - 244s 2s/step - loss: 20.5730 - masked_acc: 8.1453e-05 - masked_loss: 20.5780 - val_loss: 21.1443 - val_masked_acc: 0.0000e+00 - val_masked_loss: 21.1778
Epoch 3/3
119/119 [==============================] - 212s 2s/step - loss: 20.7125 - masked_acc: 0.0000e+00 - masked_loss: 20.7103 - val_loss: 21.0854 - val_masked_acc: 0.0000e+00 - val_masked_loss: 21.1121


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:

# model.save(f'/content/drive/MyDrive/cap-gen/{datetime.datetime.now()}-{EPOCHS}.tf')

pred = model.predict(test_data.take(1))
# print(pred.shape)

# start_token_id = word_to_id('startseq') 
# end_token_id = word_to_id('endseq') 



# aa = generate_caption(random_image_path, model, tokenizer)
# print(aa)


1/1 [==============================] - 6s 6s/step


In [22]:
pred.shape

(8, 50, 7653)

In [23]:
from preprocessing import tokens_to_text

In [28]:

START_TOKEN = 'startseq'
END_TOKEN = 'endseq'


In [29]:
def tokens_to_text(tokens):
    words = id_to_word(tokens)
    result = tf.strings.reduce_join(words, axis=-1, separator=' ')
    result = tf.strings.regex_replace(result, f'^ *{START_TOKEN} *', '')
    result = tf.strings.regex_replace(result, f' *{END_TOKEN} *$', '')
    return result


In [37]:
predd = tf.argmax(pred, axis = -1)
predd.shape

TensorShape([8, 50])

In [38]:
rr = tokens_to_text(predd)

In [39]:
rr.shape

TensorShape([8])

In [34]:
for (img, txt_in), txt_out in test_data.take(1):
    print(img.shape)

(8, 2048)


In [35]:
test.head()

,image_path,comment
115745,input/flickr30k/images/4761236916.jpg,startseq A dark haired child in swimwear laugh...
97590,input/flickr30k/images/4388731466.jpg,startseq Two dark haired men in orange shirts ...
114665,input/flickr30k/images/4745027152.jpg,startseq A young lady is colorfully dressed in...
9786,input/flickr30k/images/1557838421.jpg,startseq Two dogs wrestle with each other with...
28360,input/flickr30k/images/2414352262.jpg,startseq young girl in winter coat jumping off...


In [40]:
rr

<tf.Tensor: shape=(8,), dtype=string, numpy=
array([b'power power power power power power power power power power power power power power power arts arts expressing power conducts power arts power besides learn rolling arts power arts power impeach rolling rolling server trudging power releasing besides lightly rolling trudging impeach conducts wardrobe power power impeach impeach power besides',
       b'power power power power power power power power power power power power power power power power arts expressing power conducts power arts power besides teams rolling arts power arts power impeach rolling wardrobe server trudging power couples besides lightly rolling trudging impeach rolling wardrobe power power impeach impeach power besides',
       b'power power power power power power power power power power power power power power power arts arts expressing power conducts power arts power besides power rolling arts power arts power impeach conducts rolling server rolling power besi

In [43]:
import matplotlib.pyplot as plt
from preprocessing import load_images_now

In [44]:
start_token = START_TOKEN
end_token = END_TOKEN

In [45]:
# def generate_caption(image_path, model, tokenizer, beam_size=5):
#     ii = plt.imread(image_path)
#     plt.imshow(ii)

#     features = load_images_now(image_path)
#     features = tf.reshape(features, (1, features.shape[0]))

#     beams = [[start_token]] * beam_size

#     for _ in range(MAX_LEN):
#         for i, beam in enumerate(beams):
#             sequence = tokenizer([beam.decode('utf-8')])
#             sequence = tf.pad(sequence, [[0, 0], [0, MAX_LEN - tf.shape(sequence)[1]]])

#             logits = model.predict([features, sequence], verbose = 0)
#             next_word_idx = tf.argmax(logits, axis=-1)

#             next_word = id_to_word(next_word_idx.numpy()[0])

#             beams[i].append(next_word)

#             beams.sort(key=lambda x: x[-1], reverse=True)

#             beams = beams[:beam_size]

#         caption = beams[0]
#     return ' '.join(caption[1:])

In [61]:
def generate_caption(image_path):

  # Load the image and resize it to 224x224 pixels.
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=CHANNELS)
  image = tf.image.resize(image, (224, 224))

  # Get the image features from the ResNet50V2 model.
  image_features = resnet(image)

  # Get the start and end tokens.
  start_token = tf.constant([VOCAB_SIZE - 1], dtype=tf.int32)
  end_token = tf.constant([VOCAB_SIZE], dtype=tf.int32)

  # Initialize the decoder state.
  decoder_state = encoder.initialize_state(image_features)

  # Initialize the output sequence.
  output_sequence = []

  # Generate the caption.
  for i in range(MAX_LEN):

    # Get the next word.
    next_word_logits, decoder_state = decoder(image_features, decoder_state)
    next_word = tf.argmax(next_word_logits, axis=1)

    # If the next word is the end token, stop generating.
    if next_word == end_token:
      break

    # Add the next word to the output sequence.
    output_sequence.append(next_word)

  # Convert the output sequence to a string.
  output_caption = tokenizer.inverse_transform(output_sequence)

  return output_caption

In [59]:
random_img_path = test['image_path'].sample(1).values[0]

In [60]:
rr = generate_caption(random_img_path, model, tokenizer)
rr

ValueError: ignored